<a href="https://colab.research.google.com/github/Paul33333/ORPO/blob/main/%E6%97%A0%E9%9C%80%E5%8F%82%E8%80%83%E6%A8%A1%E5%9E%8B%E7%9A%84%E5%8D%95%E4%B8%80%E6%95%B4%E4%BD%93%E5%81%8F%E5%A5%BD%E4%BC%98%E5%8C%96%E7%AE%97%E6%B3%95%EF%BC%9A%E3%80%90ORPO%E3%80%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1、导入相关包和预训练的基础大模型**

In [1]:
!pip install accelerate datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
#将模型下载到本地环境后，加载模型
#pip install transformers==4.37.0 首先升级transformers到4.37.0版本及之后
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np
device = "cuda" # the device to load the model onto
model_path = "Qwen/Qwen1.5-0.5B"  #根据自身具备的算力条件进行自适应更改，如切换为1.5B、7B甚至更大，本处我们照顾在消费级显卡上进行全参微调，选择最小的0.5B版本

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
#查看显存占用
!nvidia-smi

Sat Mar 23 02:00:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0              41W / 300W |   1546MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### **1.1、重设模型的generation_config文件，以便最后对比训练前后生成式问答的效果**

In [4]:
model.generation_config

GenerationConfig {
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "max_new_tokens": 2048
}

In [5]:
model.generation_config.do_sample = True
model.generation_config.eos_token_id = [151645, 151643]
model.generation_config.pad_token_id = 151643
model.generation_config.temperature = 0.7
model.generation_config.top_p = 0.8
model.generation_config.top_k = 20
model.generation_config.transformers_version = "4.37.0"
model.generation_config.repetition_penalty = 1.05
#model.generation_config.max_new_tokens

In [6]:
model.generation_config

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "max_new_tokens": 2048,
  "pad_token_id": 151643,
  "repetition_penalty": 1.05,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}

## **2、定义模型训练超参**

In [7]:
from dataclasses import dataclass
@dataclass
class modelConfig:
    max_length:int = 550 #根据自身具备的算力条件进行自适应更改
    batch_size:int = 2
    gradient_accumulation_steps:int = 1
    alpha:float = 0.2
    log_iter:int = 500
    max_lr:float = 1e-5
    min_lr:float = 5e-7
    warmup_steps:int = 2000

## **3、导入训练数据**

In [8]:
import datasets
binarized_data = datasets.load_dataset('HuggingFaceH4/ultrafeedback_binarized')

Generating train_prefs split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating train_sft split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_prefs split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/1000 [00:00<?, ? examples/s]

### **3.1 对偏好训练数据转化成模型接受的形式：str2tokenid**


In [9]:
def tokenize_and_format(data): # data List[Dict[str, str]]
        input_ids = tokenizer.apply_chat_template(
    data, # Union[List[Dict[str, str]]
    tokenize = True,
    add_generation_prompt = False,
    #padding = True,
    truncation = True,
    max_length = modelConfig.max_length,
)

        return input_ids #tag_ids

In [10]:
## 生成偏好数据的tokenid
chosen_input_ids_list = []
#tag_ids_list = []
i = 0
while True:
    data = binarized_data['train_sft'][i]['chosen']
    data.insert(0, {"content": "You are a helpful assistant", "role": "system"})
    input_ids = tokenize_and_format(data)
    chosen_input_ids_list.append(input_ids)
    #tag_ids_list.append(tag_ids)
    i += 1
    if i%10000 == 0 or i== len(binarized_data['train_sft']):
        print(f"偏好数据已处理{i}条数据")
    if i ==25000:#len(binarized_data['train_sft']): ##先只取前25000条数据用于训练测试对比
        break
print('-'*70)
#############################################################################
## 生成不偏好数据的tokenid
rejected_input_ids_list = []
#tag_ids_list = []
i = 0
while True:
    data = binarized_data['train_sft'][i]['rejected']
    data.insert(0, {"content": "You are a helpful assistant", "role": "system"})
    input_ids = tokenize_and_format(data)
    rejected_input_ids_list.append(input_ids)
    #tag_ids_list.append(tag_ids)
    i += 1
    if i%10000 == 0 or i== len(binarized_data['train_sft']):
        print(f"非偏好数据已处理{i}条数据")
    if i ==25000:# len(binarized_data['train_sft']): ##先只取前25000条数据用于训练测试对比
        break

assert len(chosen_input_ids_list) == len(rejected_input_ids_list)  ##确保数据条数一致

偏好数据已处理10000条数据
偏好数据已处理20000条数据
----------------------------------------------------------------------
非偏好数据已处理10000条数据
非偏好数据已处理20000条数据


## **4、进行模型训练**

### **4.1 使用训练设置的超参数**

In [11]:
batch_size = modelConfig.batch_size
gradient_accumulation_steps = modelConfig.gradient_accumulation_steps
log_iter = modelConfig.log_iter
alpha = modelConfig.alpha
max_lr = modelConfig.max_lr
min_lr = modelConfig.min_lr
warmup_steps = modelConfig.warmup_steps
total_steps = len(chosen_input_ids_list)//batch_size
optimizer = torch.optim.AdamW(filter(lambda p:p.requires_grad, model.parameters()), lr=max_lr)
trainable_parameters_num = sum(p.numel() for p in filter(lambda p:p.requires_grad, model.parameters()))  ##全参微调

### **4.2 配置logging日志记录模型训练过程**

In [12]:
##配置logging
import time
#import logging
with open("/content/orpo_log.txt", "a") as my_file:
  my_file.write(f'time:{time.strftime("%Y-%m-%d, %H:%M:%S")}, batch_size:{batch_size}, trainable_parameters_num:{trainable_parameters_num}, warmup_steps:{warmup_steps}, max_lr:{max_lr}, min_lr:{min_lr}\n')
#定义一个日志记录函数
def log_call(iters, iters_average_loss, iters_average_logprobability_for_Positive, iters_average_logprobability_for_Negetive, iters_average_Log_Odds_Ratio, iters_average_Log_Odds):
  with open("/content/orpo_log.txt", "a") as my_file:
    my_file.write(f'time:{time.strftime("%Y-%m-%d, %H:%M:%S")}, iters:{iters+1}, iters_average_Loss:{iters_average_loss:.4f}, iters_average_logprobability_for_Positive:{iters_average_logprobability_for_Positive:.4f}, iters_average_logprobability_for_Negetive:{iters_average_logprobability_for_Negetive:.4f}, iters_average_Log_Odds_Ratio:{iters_average_Log_Odds_Ratio:.4f}, iters_average_Log_Odds:{iters_average_Log_Odds:.4f}\n')


### **4.3 设置具备warmup和余弦半衰期的学习率**

In [13]:
def linear_warmup(current_step, warmup_steps, max_lr):
    if current_step < warmup_steps:
        return max_lr * current_step / warmup_steps
    else:
        return max_lr

def cosine_decay(current_step, warmup_steps, total_steps, max_lr, min_lr):
    if current_step < warmup_steps:
        return linear_warmup(current_step, warmup_steps, max_lr)
    else:
        progress = (current_step - warmup_steps) / (total_steps - warmup_steps)
        decay = 0.5 * (1 + np.cos(np.pi * progress))
        return (max_lr - min_lr) * decay + min_lr

### **4.4 设置回答部分的掩码函数，用于执行仅针对回答部分才计算损失**

In [14]:
def return_answer_mask(input_ids):
  assistant_answer_mask = torch.zeros_like(input_ids)
  for i in range(input_ids.shape[0]):
        ## user部分的结尾\n: \n是<|im_end|>的下一个元素，所以有+1 【这个地方需要根据不同模型的不同聊天模版自定义更改】，关于聊天模版可阅读这篇文章：https://huggingface.co/blog/chat-templates
        i_user_end_list = [i+1 for i in torch.where(input_ids[i]==tokenizer.encode('<|im_end|>')[0])[0].tolist()[1::2]]
        ## assistant部分的结尾\n：\n是<|im_end|>的下一个元素，所以有+1 【这个地方需要根据不同模型的不同聊天模版自定义更改】
        i_assistant_end_list = [i+1 for i in torch.where(input_ids[i]==tokenizer.encode('<|im_end|>')[0])[0].tolist()[2::2]]

        if len(i_user_end_list)==len(i_assistant_end_list):
            for user_end, assistant_end in zip(i_user_end_list, i_assistant_end_list):
                assistant_answer_mask[i][user_end+3:assistant_end+1]=1 #+3的操作，【这个地方需要根据不同模型的不同聊天模版自定义更改】
        elif len(i_user_end_list)==len(i_assistant_end_list)+1==1:  ##单轮问答,且回答部分未结尾就被截断了
            assistant_answer_mask[i][i_user_end_list[0]+3:]=1  ##会把右补的padding token也标记为1，所以后面还需要再结合padding mask以过滤padding
        elif len(i_user_end_list)==len(i_assistant_end_list)+1:   ##兼顾多轮问答，且最后一轮对话的回答部分还未结尾就被截断了，此处我们只看完整回答部分的损失
            for user_end, assistant_end in zip(i_user_end_list[:-1], i_assistant_end_list):
                assistant_answer_mask[i][user_end+3:assistant_end+1]=1
        else:
            continue  ##跳出当前循环，继续下一次循环
  return assistant_answer_mask

### **4.5 对模型进行ORPO训练**

In [15]:
model.train()
train_loss_list = []
pos_prob_list = []
neg_prob_list = []
Log_Odds_Ratio_list = []
Log_Odds_list = []
model.zero_grad() ##clear gradients at the start of training
ignore_iters_count = 0
for iters in range(len(chosen_input_ids_list)//batch_size):
    ## 获取批次数据
    chosen_batch_inputids = chosen_input_ids_list[iters*batch_size:(iters+1)*batch_size]
    rejected_batch_inputids = rejected_input_ids_list[iters*batch_size:(iters+1)*batch_size]

    ## 对该批次数据进行padding,以并行计算，首先计算该批次的最大token长度
    chosen_max_dim = max([len(i) for i in chosen_batch_inputids])
    rejected_max_dim = max([len(i) for i in rejected_batch_inputids])
    ### 偏好数据padding填充
    chosen_batch_inputids_padding_list = []
    for i in range(batch_size):
        chosen_batch_inputids_padding_list.append(torch.nn.functional.pad(torch.tensor(chosen_batch_inputids[i]), (0, chosen_max_dim - len(chosen_batch_inputids[i])), mode='constant', value=model.generation_config.eos_token_id[-1]).tolist()) #右补
    chosen_batch_inputids_tensor = torch.tensor(chosen_batch_inputids_padding_list)
    ### 非偏好数据padding填充
    rejected_batch_inputids_padding_list = []
    for i in range(batch_size):
        rejected_batch_inputids_padding_list.append(torch.nn.functional.pad(torch.tensor(rejected_batch_inputids[i]), (0, rejected_max_dim - len(rejected_batch_inputids[i])), mode='constant', value=model.generation_config.eos_token_id[-1]).tolist()) #右补
    rejected_batch_inputids_tensor = torch.tensor(rejected_batch_inputids_padding_list)

    ## 构建训练数据：x->y ,下一个单词预测
    chosen_x = chosen_batch_inputids_tensor[:, :-1].to(device)
    chosen_y = chosen_batch_inputids_tensor[:, 1:].to(device)
    rejected_x = rejected_batch_inputids_tensor[:, :-1].to(device)
    rejected_y = rejected_batch_inputids_tensor[:, 1:].to(device)

    ## 构建掩码判别矩阵（paddding mask & answer_mask, padding mask用于执行对padding的token不计算损失，answer_mask用于执行仅针对回答部分才计算损失），总之，就是确认哪些tokens的logit需要"忽视"掉
    ### 【padding mask】
    chosen_padding_mask = torch.where(chosen_y==model.generation_config.eos_token_id[-1], 0, 1)
    rejected_padding_mask = torch.where(rejected_y==model.generation_config.eos_token_id[-1], 0, 1)
    ### 【answer_mask】
    chosen_assistant_answer_mask = return_answer_mask(chosen_x)
    rejected_assistant_answer_mask = return_answer_mask(rejected_x)
    ### 【paddingmask & answermask】方便使用掩码判别矩阵对logit和y进行过滤->:我们只关注【回答】部分的损失，不关注问题部分的损失
    chosen_assistant_answer_mask = (chosen_assistant_answer_mask&chosen_padding_mask)
    rejected_assistant_answer_mask = (rejected_assistant_answer_mask&rejected_padding_mask)

    if chosen_assistant_answer_mask.sum(dim=-1).min().item()==0 or rejected_assistant_answer_mask.sum(dim=-1).min().item()==0:##如果该批次里有的问答数据在数据截取时，回答部分存在未有数据的情况，那么该批次数据不再训练
      #print(f'不处理第{iters+1}批次数据')
      ignore_iters_count+=1
      continue  #跳出当前循环

    ## 执行偏好数据的模型推理，计算logits
    chosen_logits = model(chosen_x).logits
    torch.cuda.empty_cache() #清除非必要的显存占用，但会导致速度变慢

    ## 执行非偏好数据的模型推理，计算logits
    rejected_logits = model(rejected_x).logits
    torch.cuda.empty_cache() #清除非必要的显存占用，但会导致速度变慢

    """ORPO论文链接 https://arxiv.org/abs/2403.07691"""
    ## Compute Chosen_Answer_Loss，计算偏好数据的回答部分的损失, pos_loss的shape_size:[batch_size]
    pos_loss = torch.mul((torch.gather(torch.log(torch.softmax(chosen_logits, dim=-1)), dim=-1, index=chosen_y.unsqueeze(2))*(-1)).squeeze(2), chosen_assistant_answer_mask).sum(dim=-1) / chosen_assistant_answer_mask.sum(dim=-1)
    ## Calculate Log Probability, pos_prob、neg_prob的shape_size:[batch_size]
    pos_prob = torch.mul((torch.gather(torch.log(torch.softmax(chosen_logits, dim=-1)), dim=-1, index=chosen_y.unsqueeze(2))).squeeze(2), chosen_assistant_answer_mask).sum(dim=-1) / chosen_assistant_answer_mask.sum(dim=-1)
    neg_prob = torch.mul((torch.gather(torch.log(torch.softmax(rejected_logits, dim=-1)), dim=-1, index=rejected_y.unsqueeze(2))).squeeze(2), rejected_assistant_answer_mask).sum(dim=-1) / rejected_assistant_answer_mask.sum(dim=-1)
    ## Calculate log odds
    log_odds = (pos_prob - neg_prob) - (torch.log(1 - torch.exp(pos_prob)) - torch.log(1 - torch.exp(neg_prob)))
    sig_ratio = torch.nn.functional.sigmoid(log_odds)
    ratio = torch.log(sig_ratio)
    ## Calculate the Final Loss, 参考论文的方式，只是新增了梯度积累的操作
    loss = torch.nanmean(pos_loss - alpha * ratio)/(gradient_accumulation_steps)

    loss.backward()

    # Compute the learning rate for the current step
    lr = cosine_decay(iters, warmup_steps, total_steps, max_lr, min_lr)

    # Update the learning rate for the AdamW optimizer
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    if (iters+1)%gradient_accumulation_steps==0 or (iters+1)==(len(input_ids_list)//batch_size):
        optimizer.step() #update weights after gradients accumulation
        ##at last, clear gradients
        optimizer.zero_grad() #clear gradients after updating, in this case equal to model.zero_grad()

    train_loss_list.append(loss.item()*gradient_accumulation_steps)
    pos_prob_list.append(torch.nanmean(pos_prob).item())
    neg_prob_list.append(torch.nanmean(neg_prob).item())
    Log_Odds_Ratio_list.append(torch.nanmean(ratio).item())
    Log_Odds_list.append(torch.nanmean(log_odds).item())
    if (iters+1)%log_iter==0 or (iters+1)==(len(chosen_input_ids_list)//batch_size):
      print(f'time:{time.strftime("%Y-%m-%d, %H:%M:%S")}, iters:{iters+1}, last_{log_iter}_iters_average_train_Loss:{np.nanmean(train_loss_list[-log_iter:]):.4f}, last_{log_iter}_iters_average_logprobability_for_Positive:{np.nanmean(pos_prob_list[-log_iter:]):.4f}, last_{log_iter}_iters_average_logprobability_for_Negetive:{np.nanmean(neg_prob_list[-log_iter:]):.4f}, last_{log_iter}_iters_average_Log_Odds_Ratio:{np.nanmean(Log_Odds_Ratio_list[-log_iter:]):.4f}, last_{log_iter}_iters_average_Log_Odds:{np.nanmean(Log_Odds_list[-log_iter:]):.4f}') ##避免空值影响
      log_call(iters, np.nanmean(train_loss_list[-log_iter:]), np.nanmean(pos_prob_list[-log_iter:]), np.nanmean(neg_prob_list[-log_iter:]), np.nanmean(Log_Odds_Ratio_list[-log_iter:]), np.nanmean(Log_Odds_list[-log_iter:]))
print("Totally Completed!")
print(f'共计忽略{ignore_iters_count}个批次数据')

time:2024-03-23, 02:14:26, iters:1000, last_500_iters_average_train_Loss:1.5689, last_500_iters_average_logprobability_for_Positive:-1.4239, last_500_iters_average_logprobability_for_Negetive:-1.5741, last_500_iters_average_Log_Odds_Ratio:-0.7248, last_500_iters_average_Log_Odds:0.1924
time:2024-03-23, 02:19:58, iters:1500, last_500_iters_average_train_Loss:1.5248, last_500_iters_average_logprobability_for_Positive:-1.3807, last_500_iters_average_logprobability_for_Negetive:-1.5127, last_500_iters_average_Log_Odds_Ratio:-0.7202, last_500_iters_average_Log_Odds:0.1785
time:2024-03-23, 02:25:33, iters:2000, last_500_iters_average_train_Loss:1.5296, last_500_iters_average_logprobability_for_Positive:-1.3827, last_500_iters_average_logprobability_for_Negetive:-1.5136, last_500_iters_average_Log_Odds_Ratio:-0.7344, last_500_iters_average_Log_Odds:0.1599
time:2024-03-23, 02:31:02, iters:2500, last_500_iters_average_train_Loss:1.5298, last_500_iters_average_logprobability_for_Positive:-1.3820

## **5、保存模型，并推送到hub**

In [16]:
torch.save(model.state_dict(),'/content/'+model_path.split('/')[1]+'_ORPO.pth')

In [17]:
model.push_to_hub(model_path.split('/')[1]+'_ORPO')

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/928M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Yh-Paul/Qwen1.5-0.5B_ORPO/commit/f14e8ec29989f957241ce5c335053bc2ef5dae30', commit_message='Upload Qwen2ForCausalLM', commit_description='', oid='f14e8ec29989f957241ce5c335053bc2ef5dae30', pr_url=None, pr_revision=None, pr_num=None)

## **6、为了对比ORPO是否真的更有效，我们再只针对偏好数据进行SFT训练【但是不再增加惩罚项】，查看模型训练效果**

### **6.1 重新启动内核，清空显存占用**

In [ ]:
import os
import sys

def restart_program():
    python = sys.executable
    os.execl(python, python, *sys.argv)

if __name__ == "__main__":
    # Your Python code here
    # ...
    # Restart the program
    restart_program()


### **6.2 重装必要的环境和下载模型及后续【复制前面ORPO环节的代码】**

---



In [1]:
!pip install accelerate datasets
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np
device = "cuda" # the device to load the model onto
model_path = "Qwen/Qwen1.5-0.5B"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **6.3 开始模型的SFT训练**

In [2]:
model.generation_config.do_sample = True
model.generation_config.eos_token_id = [151645, 151643]
model.generation_config.pad_token_id = 151643
model.generation_config.temperature = 0.7
model.generation_config.top_p = 0.8
model.generation_config.top_k = 20
model.generation_config.transformers_version = "4.37.0"
model.generation_config.repetition_penalty = 1.05
#model.generation_config.max_new_tokens

from dataclasses import dataclass
@dataclass
class modelConfig:
    max_length:int = 550
    batch_size:int = 2
    gradient_accumulation_steps:int = 1
    alpha:float = 0.2
    log_iter:int = 500
    max_lr:float = 1e-5
    min_lr:float = 5e-7
    warmup_steps:int = 2000

import datasets
binarized_data = datasets.load_dataset('HuggingFaceH4/ultrafeedback_binarized')

def tokenize_and_format(data): # data List[Dict[str, str]]
        input_ids = tokenizer.apply_chat_template(
    data, # Union[List[Dict[str, str]]
    tokenize = True,
    add_generation_prompt = False,
    #padding = True,
    truncation = True,
    max_length = modelConfig.max_length,
)

        return input_ids #tag_ids


## 生成偏好数据的tokenid
chosen_input_ids_list = []
#tag_ids_list = []
i = 0
while True:
    data = binarized_data['train_sft'][i]['chosen']
    data.insert(0, {"content": "You are a helpful assistant", "role": "system"})
    input_ids = tokenize_and_format(data)
    chosen_input_ids_list.append(input_ids)
    #tag_ids_list.append(tag_ids)
    i += 1
    if i%10000 == 0 or i== len(binarized_data['train_sft']):
        print(f"偏好数据已处理{i}条数据")
    if i ==25000:#len(binarized_data['train_sft']):
        break
print('-'*70)
#############################################################################
## 生成不偏好数据的tokenid
rejected_input_ids_list = []
#tag_ids_list = []
i = 0
while True:
    data = binarized_data['train_sft'][i]['rejected']
    data.insert(0, {"content": "You are a helpful assistant", "role": "system"})
    input_ids = tokenize_and_format(data)
    rejected_input_ids_list.append(input_ids)
    #tag_ids_list.append(tag_ids)
    i += 1
    if i%10000 == 0 or i== len(binarized_data['train_sft']):
        print(f"非偏好数据已处理{i}条数据")
    if i == 25000:#len(binarized_data['train_sft']):
        break

assert len(chosen_input_ids_list) == len(rejected_input_ids_list)  ##确保数据条数一致

batch_size = modelConfig.batch_size
gradient_accumulation_steps = modelConfig.gradient_accumulation_steps
log_iter = modelConfig.log_iter
alpha = modelConfig.alpha
max_lr = modelConfig.max_lr
min_lr = modelConfig.min_lr
warmup_steps = modelConfig.warmup_steps
total_steps = len(chosen_input_ids_list)//batch_size
optimizer = torch.optim.AdamW(filter(lambda p:p.requires_grad, model.parameters()), lr=max_lr)
trainable_parameters_num = sum(p.numel() for p in filter(lambda p:p.requires_grad, model.parameters()))  ##全参微调

##配置logging
import time
#import logging
with open("/content/sft_log.txt", "a") as my_file:
  my_file.write(f'time:{time.strftime("%Y-%m-%d, %H:%M:%S")}, batch_size:{batch_size}, trainable_parameters_num:{trainable_parameters_num}, warmup_steps:{warmup_steps}, max_lr:{max_lr}, min_lr:{min_lr}\n')
#定义一个日志记录函数
def log_call(iters, iters_average_loss, iters_average_logprobability_for_Positive, iters_average_logprobability_for_Negetive, iters_average_Log_Odds_Ratio, iters_average_Log_Odds):
  with open("/content/sft_log.txt", "a") as my_file:
    my_file.write(f'time:{time.strftime("%Y-%m-%d, %H:%M:%S")}, iters:{iters+1}, iters_average_Loss:{iters_average_loss:.4f}, iters_average_logprobability_for_Positive:{iters_average_logprobability_for_Positive:.4f}, iters_average_logprobability_for_Negetive:{iters_average_logprobability_for_Negetive:.4f}, iters_average_Log_Odds_Ratio:{iters_average_Log_Odds_Ratio:.4f}, iters_average_Log_Odds:{iters_average_Log_Odds:.4f}\n')


def linear_warmup(current_step, warmup_steps, max_lr):
    if current_step < warmup_steps:
        return max_lr * current_step / warmup_steps
    else:
        return max_lr

def cosine_decay(current_step, warmup_steps, total_steps, max_lr, min_lr):
    if current_step < warmup_steps:
        return linear_warmup(current_step, warmup_steps, max_lr)
    else:
        progress = (current_step - warmup_steps) / (total_steps - warmup_steps)
        decay = 0.5 * (1 + np.cos(np.pi * progress))
        return (max_lr - min_lr) * decay + min_lr


def return_answer_mask(input_ids):
  assistant_answer_mask = torch.zeros_like(input_ids)
  for i in range(input_ids.shape[0]):
        ## user部分的结尾\n: \n是<|im_end|>的下一个元素，所以有+1 【这个地方需要根据不同模型的不同聊天模版自定义更改】，关于聊天模版可阅读这篇文章：https://huggingface.co/blog/chat-templates
        i_user_end_list = [i+1 for i in torch.where(input_ids[i]==tokenizer.encode('<|im_end|>')[0])[0].tolist()[1::2]]
        ## assistant部分的结尾\n：\n是<|im_end|>的下一个元素，所以有+1 【这个地方需要根据不同模型的不同聊天模版自定义更改】
        i_assistant_end_list = [i+1 for i in torch.where(input_ids[i]==tokenizer.encode('<|im_end|>')[0])[0].tolist()[2::2]]

        if len(i_user_end_list)==len(i_assistant_end_list):
            for user_end, assistant_end in zip(i_user_end_list, i_assistant_end_list):
                assistant_answer_mask[i][user_end+3:assistant_end+1]=1 #+3的操作，【这个地方需要根据不同模型的不同聊天模版自定义更改】
        elif len(i_user_end_list)==len(i_assistant_end_list)+1==1:  ##单轮问答,且回答部分未结尾就被截断了
            assistant_answer_mask[i][i_user_end_list[0]+3:]=1  ##会把右补的padding token也标记为1，所以后面还需要再结合padding mask以过滤padding
        elif len(i_user_end_list)==len(i_assistant_end_list)+1:   ##兼顾多轮问答，且最后一轮对话的回答部分还未结尾就被截断了，此处我们只看完整回答部分的损失
            for user_end, assistant_end in zip(i_user_end_list[:-1], i_assistant_end_list):
                assistant_answer_mask[i][user_end+3:assistant_end+1]=1
        else:
            continue  ##跳出当前循环，继续下一次循环
  return assistant_answer_mask


model.train()
train_loss_list = []
pos_prob_list = []
neg_prob_list = []
Log_Odds_Ratio_list = []
Log_Odds_list = []
model.zero_grad() ##clear gradients at the start of training
ignore_iters_count = 0
for iters in range(len(chosen_input_ids_list)//batch_size):
    ## 获取批次数据
    chosen_batch_inputids = chosen_input_ids_list[iters*batch_size:(iters+1)*batch_size]
    rejected_batch_inputids = rejected_input_ids_list[iters*batch_size:(iters+1)*batch_size]

    ## 对该批次数据进行padding,以并行计算，首先计算该批次的最大token长度
    chosen_max_dim = max([len(i) for i in chosen_batch_inputids])
    rejected_max_dim = max([len(i) for i in rejected_batch_inputids])
    ### 偏好数据padding填充
    chosen_batch_inputids_padding_list = []
    for i in range(batch_size):
        chosen_batch_inputids_padding_list.append(torch.nn.functional.pad(torch.tensor(chosen_batch_inputids[i]), (0, chosen_max_dim - len(chosen_batch_inputids[i])), mode='constant', value=model.generation_config.eos_token_id[-1]).tolist()) #右补
    chosen_batch_inputids_tensor = torch.tensor(chosen_batch_inputids_padding_list)

    ### 非偏好数据padding填充
    rejected_batch_inputids_padding_list = []
    for i in range(batch_size):
        rejected_batch_inputids_padding_list.append(torch.nn.functional.pad(torch.tensor(rejected_batch_inputids[i]), (0, rejected_max_dim - len(rejected_batch_inputids[i])), mode='constant', value=model.generation_config.eos_token_id[-1]).tolist()) #右补
    rejected_batch_inputids_tensor = torch.tensor(rejected_batch_inputids_padding_list)

    ## 构建训练数据：x->y ,下一个单词预测
    chosen_x = chosen_batch_inputids_tensor[:, :-1].to(device)
    chosen_y = chosen_batch_inputids_tensor[:, 1:].to(device)
    rejected_x = rejected_batch_inputids_tensor[:, :-1].to(device)
    rejected_y = rejected_batch_inputids_tensor[:, 1:].to(device)

    ## 构建掩码判别矩阵（paddding mask & answer_mask, padding mask用于执行对padding的token不计算损失，answer_mask用于执行仅针对回答部分才计算损失），总之，就是确认哪些tokens的logit需要"忽视"掉
    ### 【padding mask】
    chosen_padding_mask = torch.where(chosen_y==model.generation_config.eos_token_id[-1], 0, 1)
    rejected_padding_mask = torch.where(rejected_y==model.generation_config.eos_token_id[-1], 0, 1)
    ### 【answer_mask】
    chosen_assistant_answer_mask = return_answer_mask(chosen_x)
    rejected_assistant_answer_mask = return_answer_mask(rejected_x)
    ### 【paddingmask & answermask】方便使用掩码判别矩阵对logit和y进行过滤->:我们只关注【回答】部分的损失，不关注问题部分的损失
    chosen_assistant_answer_mask = (chosen_assistant_answer_mask&chosen_padding_mask)
    rejected_assistant_answer_mask = (rejected_assistant_answer_mask&rejected_padding_mask)

    if chosen_assistant_answer_mask.sum(dim=-1).min().item()==0  or rejected_assistant_answer_mask.sum(dim=-1).min().item()==0:##如果该批次里有的问答数据在数据截取时，回答部分存在未有数据的情况，那么该批次数据不再训练
      #print(f'不处理第{iters+1}批次数据')
      ignore_iters_count+=1
      continue  #跳出当前循环

    ## 执行偏好数据的模型推理，计算logits
    chosen_logits = model(chosen_x).logits
    torch.cuda.empty_cache() #清除非必要的显存占用，但会导致速度变慢
    model.eval()
    with torch.no_grad():
      ## 执行非偏好数据的模型推理，计算logits
      rejected_logits = model(rejected_x).logits
      torch.cuda.empty_cache() #清除非必要的显存占用，但会导致速度变慢
    model.train()

    ## 计算偏好数据的回答部分的损失, pos_loss的shape_size:[batch_size]
    """ORPO论文链接 https://arxiv.org/abs/2403.07691"""
    ## Compute Chosen_Answer_Loss，计算偏好数据的回答部分的损失, pos_loss的shape_size:[batch_size]
    pos_loss = torch.mul((torch.gather(torch.log(torch.softmax(chosen_logits, dim=-1)), dim=-1, index=chosen_y.unsqueeze(2))*(-1)).squeeze(2), chosen_assistant_answer_mask).sum(dim=-1) / chosen_assistant_answer_mask.sum(dim=-1)

    ## Calculate Log Probability, pos_prob、neg_prob的shape_size:[batch_size]
    pos_prob = torch.mul((torch.gather(torch.log(torch.softmax(chosen_logits, dim=-1)), dim=-1, index=chosen_y.unsqueeze(2))).squeeze(2), chosen_assistant_answer_mask).sum(dim=-1) / chosen_assistant_answer_mask.sum(dim=-1)
    neg_prob = torch.mul((torch.gather(torch.log(torch.softmax(rejected_logits, dim=-1)), dim=-1, index=rejected_y.unsqueeze(2))).squeeze(2), rejected_assistant_answer_mask).sum(dim=-1) / rejected_assistant_answer_mask.sum(dim=-1)
    ## Calculate log odds
    log_odds = (pos_prob - neg_prob) - (torch.log(1 - torch.exp(pos_prob)) - torch.log(1 - torch.exp(neg_prob)))
    sig_ratio = torch.nn.functional.sigmoid(log_odds)
    ratio = torch.log(sig_ratio)

    ## Calculate the Final Loss, 参考论文的方式，只是新增了梯度积累的操作
    loss = torch.nanmean(pos_loss)/(gradient_accumulation_steps)

    loss.backward()

    # Compute the learning rate for the current step
    lr = cosine_decay(iters, warmup_steps, total_steps, max_lr, min_lr)

    # Update the learning rate for the AdamW optimizer
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    if (iters+1)%gradient_accumulation_steps==0 or (iters+1)==(len(input_ids_list)//batch_size):
        optimizer.step() #update weights after gradients accumulation
        ##at last, clear gradients
        optimizer.zero_grad() #clear gradients after updating, in this case equal to model.zero_grad()

    train_loss_list.append(loss.item()*gradient_accumulation_steps)
    pos_prob_list.append(torch.nanmean(pos_prob).item())
    neg_prob_list.append(torch.nanmean(neg_prob).item())
    Log_Odds_Ratio_list.append(torch.nanmean(ratio).item())
    Log_Odds_list.append(torch.nanmean(log_odds).item())
    if (iters+1)%log_iter==0 or (iters+1)==(len(chosen_input_ids_list)//batch_size):
      print(f'time:{time.strftime("%Y-%m-%d, %H:%M:%S")}, iters:{iters+1}, last_{log_iter}_iters_average_train_Loss:{np.nanmean(train_loss_list[-log_iter:]):.4f}, last_{log_iter}_iters_average_logprobability_for_Positive:{np.nanmean(pos_prob_list[-log_iter:]):.4f}, last_{log_iter}_iters_average_logprobability_for_Negetive:{np.nanmean(neg_prob_list[-log_iter:]):.4f}, last_{log_iter}_iters_average_Log_Odds_Ratio:{np.nanmean(Log_Odds_Ratio_list[-log_iter:]):.4f}, last_{log_iter}_iters_average_Log_Odds:{np.nanmean(Log_Odds_list[-log_iter:]):.4f}') ##避免空值影响
      log_call(iters, np.nanmean(train_loss_list[-log_iter:]), np.nanmean(pos_prob_list[-log_iter:]), np.nanmean(neg_prob_list[-log_iter:]), np.nanmean(Log_Odds_Ratio_list[-log_iter:]), np.nanmean(Log_Odds_list[-log_iter:]))
print("Totally Completed!")
print(f'共计忽略{ignore_iters_count}个批次数据')

偏好数据已处理10000条数据
偏好数据已处理20000条数据
----------------------------------------------------------------------
非偏好数据已处理10000条数据
非偏好数据已处理20000条数据
time:2024-03-23, 04:31:58, iters:1000, last_500_iters_average_train_Loss:1.4231, last_500_iters_average_logprobability_for_Positive:-1.4231, last_500_iters_average_logprobability_for_Negetive:-1.5727, last_500_iters_average_Log_Odds_Ratio:-0.7254, last_500_iters_average_Log_Odds:0.1912
time:2024-03-23, 04:35:37, iters:1500, last_500_iters_average_train_Loss:1.3795, last_500_iters_average_logprobability_for_Positive:-1.3795, last_500_iters_average_logprobability_for_Negetive:-1.5107, last_500_iters_average_Log_Odds_Ratio:-0.7205, last_500_iters_average_Log_Odds:0.1772
time:2024-03-23, 04:39:17, iters:2000, last_500_iters_average_train_Loss:1.3818, last_500_iters_average_logprobability_for_Positive:-1.3818, last_500_iters_average_logprobability_for_Negetive:-1.5109, last_500_iters_average_Log_Odds_Ratio:-0.7360, last_500_iters_average_Log_Odds:0.1581
ti

### **6.4 保存SFT模型并推送hub**

In [3]:
torch.save(model.state_dict(),'/content/'+model_path.split('/')[1]+'_SFT.pth')
model.push_to_hub(model_path.split('/')[1]+'_SFT')

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/928M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Yh-Paul/Qwen1.5-0.5B_SFT/commit/093cef05c7251306bc5b952da75e9e8df5e7b47a', commit_message='Upload Qwen2ForCausalLM', commit_description='', oid='093cef05c7251306bc5b952da75e9e8df5e7b47a', pr_url=None, pr_revision=None, pr_num=None)

## **7、ORPO VS SFT 训练后的模型生成效果对比**

In [ ]:
## 先重启内核并安装必要的环境
import os
import sys

def restart_program():
    python = sys.executable
    os.execl(python, python, *sys.argv)

if __name__ == "__main__":
    # Your Python code here
    # ...
    # Restart the program
    restart_program()

In [1]:

!pip install accelerate datasets
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np
device = "cuda" # the device to load the model onto
model_path = "Qwen/Qwen1.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **7.1 加载ORPO和SFT各自训练后的模型**

In [2]:
torch.cuda.empty_cache()
model_sft = AutoModelForCausalLM.from_pretrained(
    'Yh-Paul/Qwen1.5-0.5B_SFT',
    torch_dtype="auto",
    device_map="auto"
)

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/928M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

In [3]:
torch.cuda.empty_cache()
model_orpo = AutoModelForCausalLM.from_pretrained(
    'Yh-Paul/Qwen1.5-0.5B_ORPO',
    torch_dtype="auto",
    device_map="auto"
)

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/928M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

### **7.2、测试模型效果对比**

In [5]:
### 设置近似贪心解码，以在对比时，避免随机性导致的效果偏差
model_orpo.generation_config.temperature = 0.1
model_sft.generation_config.temperature = 0.1
model_orpo.generation_config.top_p = 0.3
model_sft.generation_config.top_p = 0.3
model_orpo.generation_config.top_k = 5
model_sft.generation_config.top_k = 5

In [6]:
while True:
  prompt = input('请输入你的问题：')
  messages = [
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt}
  ]

  ##使用qwen1.5系列的聊天模板进行“封装”（补充【特殊的token】进入上下文中）， 这里需要特别指出，每个模型的聊天模型都不一样， 需要特别留意， 参考阅读：https://huggingface.co/blog/chat-templates
  text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )
  #print(text)

  ##对text进行tokenizer编码，转化为token id 及 attention_mask
  model_inputs = tokenizer([text], return_tensors="pt").to(device)

  print('-'*70)
  print('SFT微调训练后的模型回答：\n')
  ##将token id送进去原模型model进行续写生成回答
  sft_generated_ids = model_sft.generate(
      model_inputs.input_ids,
      max_new_tokens=1000
  )
  ## 获得回答部分的token id
  sft_generated_ids = [
      output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, sft_generated_ids)
  ]
  ## 解码：tokenid2token
  sft_response = tokenizer.batch_decode(sft_generated_ids, skip_special_tokens=True)[0]
  print(sft_response)
  torch.cuda.empty_cache()

  print('-'*70)

  print('ORPO微调训练后的模型回答：\n')
  ##将token id送进去原模型model进行续写生成回答
  orpo_generated_ids = model_orpo.generate(
      model_inputs.input_ids,
      max_new_tokens=1000
  )
  ## 获得回答部分的token id
  orpo_generated_ids = [
      output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, orpo_generated_ids)
  ]
  ## 解码：tokenid2token
  orpo_response = tokenizer.batch_decode(orpo_generated_ids, skip_special_tokens=True)[0]
  print(orpo_response)
  torch.cuda.empty_cache()
  print('#'*100)
  print('\n\n')

请输入你的问题：你是？
----------------------------------------------------------------------
SFT微调训练后的模型回答：

你好，我是AI助手。
----------------------------------------------------------------------
ORPO微调训练后的模型回答：

你好，我是AI助手。
####################################################################################################



请输入你的问题：爸爸重要还是妈妈重要
----------------------------------------------------------------------
SFT微调训练后的模型回答：

这个问题需要根据具体情况进行判断。一般来说，父母和子女之间的关系是相互依存、相互支持的，因此在家庭中，父母和子女的关系非常重要。但是，每个人的情况不同，例如年龄、性别、职业、经济状况等，也会对父母和子女的关系产生影响。
如果一个家庭中有多个孩子，那么每个孩子都可能有自己的需求和期望，而父母则需要承担起照顾孩子的责任。同时，父母也需要关注孩子的成长和发展，为他们提供良好的教育和环境，以帮助他们成为独立、自信、有责任感的人。
因此，在家庭中，父母和子女的关系是非常重要的，但并不是绝对的。在某些情况下，父母可能会更关心自己的健康和生活，而忽略了子女的需求和感受。在这种情况下，子女也应该尊重父母的意愿和选择，与父母保持良好的沟通和合作，共同面对家庭中的挑战和困难。
----------------------------------------------------------------------
ORPO微调训练后的模型回答：

这个问题需要根据具体情况进行判断。一般来说，父母和子女之间的关系是相互依存、相互支持的，因此在家庭中，父母和子女的关系非常重要。但是，每个人的情况不同，例如年龄、性别、职业、经济状况等等，这些因素都会影响到父母和子女之间的关系。因此，无法简单地回答这个问题，因为每个人的情况都不同。
##################

KeyboardInterrupt: Interrupted by user